In [1]:
!rm -rf sample_data

# Melspectro Hop_size 512, shape 1249x128
#!wget http://dept-info.labri.fr/~hanna/ProjetClassif/melspectro_songs_train.pickle
#!wget http://dept-info.labri.fr/~hanna/ProjetClassif/melspectro_genres_train.pickle

#!wget http://dept-info.labri.fr/~hanna/ProjetClassif/melspectro_songs_test.pickle
#!wget http://dept-info.labri.fr/~hanna/ProjetClassif/melspectro_filenames_test.pickle

# Melspectro Hop_size 2048, shape 313x128
!wget http://dept-info.labri.fr/~hanna/ProjetClassif/melspectro_songs_train_new.pickle
!wget http://dept-info.labri.fr/~hanna/ProjetClassif/melspectro_genres_train_new.pickle

!wget http://dept-info.labri.fr/~hanna/ProjetClassif/melspectro_songs_test_new.pickle
!wget http://dept-info.labri.fr/~hanna/ProjetClassif/melspectro_filenames_test.pickle


--2022-12-21 11:02:13--  http://dept-info.labri.fr/~hanna/ProjetClassif/melspectro_songs_train_new.pickle
Resolving dept-info.labri.fr (dept-info.labri.fr)... 147.210.9.83, 2001:660:6101:404::2:80
Connecting to dept-info.labri.fr (dept-info.labri.fr)|147.210.9.83|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://dept-info.labri.fr/~hanna/ProjetClassif/melspectro_songs_train_new.pickle [following]
--2022-12-21 11:02:14--  https://dept-info.labri.fr/~hanna/ProjetClassif/melspectro_songs_train_new.pickle
Connecting to dept-info.labri.fr (dept-info.labri.fr)|147.210.9.83|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1280445598 (1.2G)
Saving to: ‘melspectro_songs_train_new.pickle’

melspectro_songs_tr 100%[===================>]   1.19G  4.98MB/s    in 2m 57s  

2022-12-21 11:05:11 (6.89 MB/s) - ‘melspectro_songs_train_new.pickle’ saved [1280445598/1280445598]

URL transformed to HTTPS due to an HSTS policy
--2022-12

In [2]:
import pickle

_songs=pickle.load(open('melspectro_songs_train_new.pickle','rb'))
_genres=pickle.load(open('melspectro_genres_train_new.pickle','rb'))

print(_songs.shape, _genres.shape)
xtrain= _songs
ytrain= _genres

TRACK = 2 # par exemple

#one song
print(_songs[TRACK].shape)
# genre : one hot encoding
print(_genres[TRACK])


_songs=pickle.load(open('melspectro_songs_test_new.pickle','rb'))
_track_id=pickle.load(open('melspectro_filenames_test.pickle','rb'))

print(_songs.shape, _track_id.shape)

TRACK = 2 # par exemple

#one song
print(_songs[TRACK].shape)
# genre : one hot encoding
print(_track_id[TRACK])



(3995, 313, 128) (3995, 8)
(313, 128)
[0. 0. 1. 0. 0. 0. 0. 0.]
(4002, 313, 128) (4002,)
(313, 128)
000190


## **Deep learning**

In [3]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras import datasets, layers, models
import numpy as np
import pandas as pd
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau

In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(xtrain, ytrain, test_size=0.30)

In [6]:
img_rows, img_cols = x_train.shape[1], x_train.shape[2]
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = x_train.shape[1:]

print('x_train.shape=', x_train.shape)
print('x_test.shape=', x_test.shape)
print('input_shape=',input_shape)

x_train.shape= (2796, 313, 128, 1)
x_test.shape= (1199, 313, 128, 1)
input_shape= (313, 128, 1)


In [7]:
padding = "same"
epochs=50
batch_size= 5

model = Sequential()



model.add(Conv2D(32 ,input_shape=input_shape , kernel_size=(3,3), strides=1, padding='same', activation='relu'))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(32, kernel_size=(3,3), strides=1, padding='same', activation='relu'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0))

model.add(Conv2D(64 ,input_shape=input_shape , kernel_size=(3,3), strides=1, padding='same', activation='relu'))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(64, kernel_size=(3,3), strides=1, padding='same', activation='relu'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0))

model.add(Conv2D(128 ,input_shape=input_shape , kernel_size=(3,3), strides=1, padding='same', activation='relu'))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(128, kernel_size=(3,3), strides=1, padding='same', activation='relu'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=epochs, batch_size=batch_size, callbacks=[earlyStopping, mcp_save])

Epoch 1/50
560/560 [==============================] - 27s 29ms/step - loss: 2.0995 - accuracy: 0.2711 - val_loss: 1.7001 - val_accuracy: 0.4103
Epoch 2/50
560/560 [==============================] - 14s 25ms/step - loss: 2.0501 - accuracy: 0.2500 - val_loss: 1.9131 - val_accuracy: 0.3136
Epoch 3/50
560/560 [==============================] - 14s 26ms/step - loss: 1.8711 - accuracy: 0.3119 - val_loss: 1.8936 - val_accuracy: 0.3561
Epoch 4/50
560/560 [==============================] - 15s 28ms/step - loss: 1.9343 - accuracy: 0.2722 - val_loss: 1.7822 - val_accuracy: 0.3528
Epoch 5/50
560/560 [==============================] - 15s 27ms/step - loss: 1.8208 - accuracy: 0.3162 - val_loss: 3.0034 - val_accuracy: 0.2911
Epoch 6/50
560/560 [==============================] - 15s 26ms/step - loss: 1.8036 - accuracy: 0.3323 - val_loss: 1.6736 - val_accuracy: 0.4320
Epoch 7/50
560/560 [==============================] - 16s 28ms/step - loss: 1.7214 - accuracy: 0.3705 - val_loss: 1.6287 - val_accuracy:

In [8]:
# Evaluate the model
print("Evaluation data")
results = model.evaluate(x_test, y_test, batch_size=5)
print("test loss, test acc:", results)

Evaluation data
240/240 [==============================] - 2s 8ms/step - loss: 1.5609 - accuracy: 0.4754
test loss, test acc: [1.5609208345413208, 0.47539615631103516]


In [9]:
songs = _songs.reshape(_songs.shape[0], img_rows, img_cols, 1)
predicted_y = model.predict(songs)

126/126 [==============================] - 6s 35ms/step


In [10]:
predicted_y = np.argmax(predicted_y,axis=1)
print(predicted_y.shape)

(4002,)


In [11]:
import csv


with open('test.csv', 'w', newline='') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        
        spamwriter.writerow(['track_id']+['genre_id'])
        
        for ind,x in np.ndenumerate(_track_id):
          csvfile.write("%s,%s\n"%(x,predicted_y[ind]+1))
        csvfile.write("%s,%s\n"%('59684',2))
        csvfile.write("%s,%s\n"%('98565',2))
        csvfile.write("%s,%s\n"%('98568',2))
        csvfile.write("%s,%s\n"%('98569',2))
        csvfile.write("%s,%s\n"%('98571',2))
        csvfile.write("%s,%s\n"%('98559',2))